In [16]:
import lxml.etree as ET
from words import tei
import xmltodict as xd

In [17]:
data = tei.TeiReader('words/data/data.xml')

In [18]:
conceptschemes = ['word_type', 'pos', 'gender', 'subc', 'usg', 'morph_type']

In [35]:
def create_word(legacy_id, lang, orth, wordtype):
    lang_scheme,_ = SkosConceptScheme.objects.get_or_create(dc_title='language')
    lang_concept,_ = SkosConcept.objects.get_or_create(pref_label=lang)
    lang_concept.scheme.set([lang_scheme])
    word_type_scheme,_ = SkosConceptScheme.objects.get_or_create(dc_title='word type')
    word_type,_ = SkosConcept.objects.get_or_create(pref_label=wordtype)
    word_type.scheme.set([word_type_scheme])
    new_word,_ = Word.objects.get_or_create(
        legacy_id=legacy_id,
        word_type=word_type,
        orth=orth,
        language=lang_concept
    )
    return new_word    

In [36]:
hansi = create_word('1234', 'french', 'SUMSI', 'text_word')

In [21]:
def add_concept_as_fk(objectinstance, conceptschemes, valuelist, scheme='scheme', pref_label='pref_label'):
    """adds concepts to objects"""
    objects = []
    for x in conceptschemes:
        if valuelist.get(scheme) == x:
            label = valuelist.get(pref_label, 'no {} provided'.format(x))
            concept,_ = SkosConcept.objects.get_or_create(
                    pref_label=label
                )
            concept.scheme.add(
                    SkosConceptScheme.objects.get_or_create(dc_title=x)[0]
                )
            setattr(objectinstance, x, concept)
            objectinstance.save()
            objects.append(objectinstance)
    return objects

In [22]:
for x in conceptschemes:
    SkosConceptScheme.objects.get_or_create(
        dc_title=x
    )

In [23]:
entries = data.find_elements('entry')

In [64]:
for x in entries['hits']:
    entry = (xd.parse(ET.tostring(x)))['entry']
    legacy_id = entry.get('@xml:id', 'NO ID')
    forms = entry['form']
    if isinstance(forms, list):
        for form in forms:
            if form.get('@type') == 'lemma':
                temp_word = create_word(
                    entry.get('@xml:id', 'NO ID'),
                    form['orth'].get('@xml:lang', 'no lang'),
                    form['orth'].get('#text', 'NO TEXT'),
                    'lemma'
                )
            elif form.get('@type') == 'variant':
                temp_variant = create_word(
                    entry.get('@xml:id', 'NO ID'),
                    form['orth'].get('@xml:lang', 'no lang'),
                    form['orth'].get('#text', 'NO TEXT'),
                    'variant'
                )
                temp_word.has_form.add(temp_variant)
        temp_word.save()
    else:
        form = forms
        temp_word = create_word(
                    entry.get('@xml:id', 'NO ID'),
                    form['orth'].get('@xml:lang', 'no lang'),
                    form['orth'].get('#text', 'NO TEXT'),
                    'lemma'
                )
        if form.get('form'):
            form = form.get('form')
            if isinstance(form, list):
                for form in form:
                    if form.get('@type') == 'lemma':
                        temp_word = create_word(
                            entry.get('@xml:id', 'NO ID'),
                            form['orth'].get('@xml:lang', 'no lang'),
                            form['orth'].get('#text', 'NO TEXT'),
                            'lemma'
                        )
            else:
                temp_variant = create_word(
                        entry.get('@xml:id', 'NO ID'),
                        form['orth'].get('@xml:lang', 'no lang'),
                        form['orth'].get('#text', 'NO TEXT'),
                        'variant'
                    )
                temp_word.has_form.add(temp_variant)
    senses = entry.get('sense', 'no sense')
    if isinstance(senses, list):
        for sense in senses:
            try:
                temp_sense = create_word(
                    entry.get('@xml:id', 'NO ID'),
                    cit.get('@xml:lang', 'NO TEXT'),
                    cit.get('quote', 'NO TEXT'),
                    cit.get('@type', 'no cit type')
                )
                temp_word.has_sense.add(temp_sense)
                temp_word.save()
            except:
                pass
        
    else:
        senses = senses['cit']
        for cit in senses:
            try:
                temp_sense = create_word(
                    entry.get('@xml:id', 'NO ID'),
                    cit.get('@xml:lang', 'NO TEXT'),
                    cit.get('quote', 'NO TEXT'),
                    cit.get('@type', 'no cit type')
                )
                temp_word.has_sense.add(temp_sense)
                temp_word.save()
            except:
                pass
                
    try:
        gramtypes = entry['gramGrp']
        if not isinstance(entry.get('gramGrp'), list):
            if isinstance(entry['gramGrp'].get('gram'), list):
                for gram in entry['gramGrp'].get('gram'):
                    if not isinstance(gram, str):
                        new = add_concept_as_fk(temp_word, conceptschemes, gram, scheme='@type', pref_label='#text')
    except KeyError:
        pass
    temp_word.save()
    print(legacy_id)

ams_001
ibaadhiiya_001
ibliis_001
izhtihaad_001
ahrash_001
ahraf_001
ahna_001
adghaal_001
adhaan_001
arbiitr_001
arnib_001
azcaar_001
staahal_001
istiftaa_001
l_islaam_001
islaahiiya_001
atrish_001
l_ifriiqi_001
ifriiqya_001
iftaar_001
lafca_001
aqaarib_001
akiid_001
almaanya_001
iimaam_001
aamaan_001
amaan_001
ambuuba_001
ummiima_001
aamiin_003
entrit_001
intisaar_001
nsaab_001
ansaar_001
ingiltraa_001
l_ingliiziiya_001
uul_aams_002
uumu_001
iidaam_001
iisaar_003
iimiin_002
ay_001
baatrii_001
baar_001
baarbu_001
baarhit_l_uula_001
baas_002
baasta_001
b_ism_illaah_001
baasha_001
baasa_001
baacit_001
baac_w_shraa_001
baak_001
baal_003
baalik_002
baandi_001
baahi_001
baahii_shi_001
bay_001
babbaghayyu_001
bahh_001
bahbuuh_001
bahhaar_001
bahhar_001
baxshiish_001
baxnaas_001
bxuur_001
braavu_001
barbri_001
burzhwaazi_001
barraad_001
brid_001
birdaan_001
barsha_001
birshni_001
burt_001
burtaal_001
burghul_001
braq_001
barqish_001
barka_002
barkuun_001
barmiil_001
barnuus_001
bruuduu_001
ba

samfuunya_001
sandaal_001
sandir_001
sandriya_001
sandwitsh_001
swaak_001
suurdi_001
suuri_001
suus_001
suuq_001
suum_001
sawwit_001
sawwa_001
swiiglaas_001
siidii_001
b_is_siif_001
siigaaru_001
simaan_001
sayyib_001
shaab_002
shaarib_001
shaas_001
shaashiiya_001
shaashiiya_stambuuli_001
shaat_001
shoombrǟr_001
shaamiiya_001
shaanti_001
shaayb_001
shaayx_001
shabaab_001
shbaab_001
shibt_001
shabka_002
shtaa_001
shham_001
shidda_001
shraab_001
shraab_ahmar_001
sharq_001
sharqi_001
sharka_001
shriif_002
shirka_002
shashsha_001
shutt_001
shatha_001
shtar_001
shatranzh_001
shacb_001
shacbaata_001
shacbaan_001
shacbi_001
shcuul_001
shciir_001
shfaa_0012
shfaa_001
shaqq_001
shaqshaq_001
shaqq_002
shaqlib_001
shkaarit_il_buul_001
shukran_001
shakshuuka_001
shikkaal_001
shakkil_001
shuklaata_001
shlaaka_001
shilbuuq_001
shilha_001
shalta_001
shlaaghim_001
shlaq_001
shilla_001
shallit_001
shalliq_001
shallik_001
shallil_001
shmaata_001
shamaal_001
shmaayti_001
shiminiiya_001
shanshaana_001
shaa

mishhaah_001
mashduud_001
mashriq_001
musht_001
mashta_001
mshallit_001
mshammis_001
mishwaar_001
mshuushit_001
mshuuki_001
mshuum_001
msut_001
msaddir_002
msalha_001
maslha_001
musli_002
musawwra_001
musawwir_001
mdharrih_001
madhmadh_001
mtabbis_001
matbca_001
matbuuc_001
mtarqa_001
mtahhir_001
mtawwil_001
mtiiqir_001
mucaaridh_001
mucaardha_001
muctmid_001
mcadhdhib_001
macrfa_001
macruuf_001
mcassil_001
mcallaq_001
mcallim_001
il_maghrib_001
mughrib_001
mgharfa_001
mughzaal_001
maghshuush_001
mughyaar_001
maftuul_001
mifti_001
mfasser_001
mfashshix_001
mfaqqis_001
mfawwah_001
muqaabil_001
muqaabla_001
maqaam_001
maqruudh_001
mqazziz_001
mqass_001
mqassar_001
maqsuus_002
maqtuuc_001
maqqas_001
maqfuul_001
mqallac_001
mqallaq_001
muqli_001
mqawwis_001
mgarrac_001
magruuna_001
makbuub_001
mkattif_001
mukhla_002
mkashshix_002
makluub_001
mkammil_001
makyizh_001
maakiina_001
mlaa_001
mlaak_001
malak_001
mlaawi_001
mulhid_001
malhfa_001
malsuuqa_001
mallih_001
milliim_001
mluuxiiya_001
m

rbah_001
shrab_001
dxal_001
qcad_001
xrazh_001
rqad_001
ftar_001
qtal_001
hrab_001
xdim_001
fhim_001
zhbid_001
ktib_001
kmil_001
skun_001
lcab_001
bcat_001
hbat_001
rzhac_001
shar_001
hall_001
habb_001
fadd_001
shadd_001
hass_001
sabb_001
wzin_001
wqif_001
daar_002
shaaf_001
kaan_001
faat_001
zaar_001
qaam_001
dhaaq_001
maat_001
caam_001
qaal_001
faaq_001
shaax_001
zhaab_001
zaad_001
baac_001
baat_001
naam_001
xaaf_001
mshaa_001
kraa_001
shraa_001
zhraa_001
kwaa_001
ctaa_001
bdaa_001
qraa_001
lqaa_001
nsaa_001
zhaa_001
raa_001
daarizh_001
caaqid_001
caaqil_001
qaaris_001
naadizh_001
saahil_001
ghaariq_001
ghaamiq_001
kaasah_001
yaabis_001
shaayah_001
taayib_001
shaaish_001
caali_001
haaz_001
ghnii_001
trii_001
hayy_001
taiyib_001
zhiiyid_001
mliih_001
ndhiif_001
bciid_001
dhriif_001
ktiir_001
dhciif_001
qsiir_001
tqiil_001
smiin_001
qliil_001
xshiin_001
rziin_001
criidh_001
rkiik_001
twiil_001
ltiif_001
rimz_001
hurr_001
bliid_001
cdhiim_001
fqiir_001
bxiil_001
zciim_001
sciid_001
zhaa

dhcuf_001
qrub_001
scub_001
ciin_002
tlaatiin_001
arbciin_001
sittiin_001
tmaaniin_001
tisciin_001
tlaata_myaa_001
arbca_myaa_001
sitta_myaa_001
sabca_myaa_001
tmaanya_myaa_001
tisca_myaa_001
piisiin_001
il_xaarizh_001
talyaani_002
cadhdh_001
suura_002
haak_001
garsoo_001
il_001
f_il_caada_001
messaazh_001
ingliizi_001
ingliizi_002
uurubba_001
uurubbi_001
uurubbi_002
hziin_001
zhaac_001
kahh_001
radd_001
cadd_001
daqq_001
dharr_001
iimaayl_001
tniiya_001
dars_001
stadca_001
rtaah_001
is_snaa_001
tall_002
cshaa_001
waqt_illi_001
hayya_003
yacmil_taaliifuun_001
ciid_miilaad_001
yitfarrizh_ca_t_talfza_001
qadd_qadd_001
qlaaybi_001
qaayla_001
kaart_poostaal_001
maatsh_002
maatsh_kuura_001
qdhaa_001
zhraa_003
ghlaa_001
biskwiit_001
siikuunda_001
macna_001
faaks_001
garn_001
qism_001
mkassar_001
kiif_il_caada_001
nxal_001
wzhac_001
xass_001
xuux_001
baanaan_001
mishmaash_001
burdgaan_001
rummaan_001
qrac_001
gannaariiya_001
krumb_001
bruuklu_001
bruuklu_axdhar_001
bitnzhaan_001
habb_mluuk_00

...

zhurra_001
hazhzhaala_001
billaarizh_001
tkuurizh_001
xyaana_001
tkabbir_001
calfa_001
tashwiish_001
faashil_001
zummaara_002
imkaaniiya_001
masduud_001
taxbiish_001
wasiila_001
muqtanac_002
tsaamah_001
ghlaaf_001
ghallif_001
xuffaash_001
xsaara_001
tcallaq_001
txaatir_001
slah_001
asmar_001
tarkiiz_001
riizhim_001
tsall_001
baahit_001
mkastim_001
mdammar_001
tzharbac_001
tlazz_001
xmizh_001
xuttiifa_001
taclib_001
raas_maal_001
maadhi_001
naafih_001
naasib_001
rakkib_001
washwish_001
iscaaf_001
ghamm_002
zaghwaan_001
sfaaqis_001
maatir_001
ardh_001
rhaa_002
tbarna_002
hazzaar_001
waasta_001
wassit_001
nifaaq_001
hattam_001
sallaah_001
zhhannim_001
klaas_001
zaciim_001
gaawriiya_001
mhaff_001
mihna_001
czuuza_001
giinya_001
ghalla_002
makka_001
dallil_001
cadaala_001
shar_002
xmazh_001
habqa_001
niika_001
fahm_001
wasf_001
hafdh_001
biic_001
zhary_001
bani_001
mashy_001
xazy_001
safu_001
zahu_001
shaky_001
zhay_001
mlaacbi_001
zhaayza_001
twaqqac_001
mumattil_001
nuuma_001
ntaq_001
can

xfayyif_001
qsayyir_001
zghayyir_001
qrayyib_001
rkayyik_001
rzayyin_001
tqayyil_001
bsayyit_001
hwayyidh_001
mrayyidh_001
cerayyidh_001
ndhayyif_001
blayyid_001
rxayyis_001
dhrayyif_001
cesayya_001
sahhah_001
laabis_001
tsaabiq_001
munaafsa_001
shiifa_001
shriiba_001
nxiila_001
bsiila_001
hwiizha_001
bayyib_001
effayyim_001
karkadan_001
uzhra_001
whayyid_001
xwayyit_001
frayshik_001
klayyim_001
wsayyif_001
ytayyim_001
cedhiima_001
wxayt_001
mrayt_001
hwiira_001
twiisa_001
friida_001
tbiila_001
qeciida_001
smiisha_001
ghwiiba_001
nwiiga_001
swiica_001
shkiira_001
wriida_001
wriiqa_001
wiida_001
tkabb_001
tfassir_001
diploom_001
riiska_001
ibtidaa_i_001
hansha_001
czuuz_001
msaat_001
carsa_001
fattish_001
mufattish_001
mitfattin_001
tamarrud_001
akkid_001
bashaaca_001
sharci_001
qbiih_001
saakin_001
saakin_002
dhidd_001
mcattal_001
qmac_001
tfalsif_001
tkarkib_001
mfarrit_001
nidhaam_001
mawqac_001
maydaan_001
khuuliiya_001
kaamuufla_001
fiiksa_001
slayyim_001
xmayyis_001
cezayyiz_001
k

skiikna_001
dziizna_001
driibka_001
shliilfa_001
cliilsha_001
ksiikis_001
mniigla_001
briimla_001
birriima_001
duulaash_001
farmla_001
zhriirya_001
swiirya_001
xaddaam_002
saariq_002
tbiiba_001
labbis_001
sabsi_001
aaxra_001
magduud_001
min_hakka_001
fuuq_shwayya_001
sriiwil_001
msiikin_001
sriidik_001
sniidiq_001
ctiiris_001
mriiwil_001
msiirin_001
mniidla_001
kciibra_001
mriifac_001
msiikin_002
mxiizin_001
mkiitib_001
mfiitih_001
mghiirfa_001
mtiirqa_001
mhiirma_001
tsiikra_001
mqiisra_001
dhwiifra_001
dwiirya_001
naqqil_001
caarif_002
tgarbiic_001
telbis_001
qaahir_001
tsuuqiit_001
daxlaani_001
shahhaar_001
msammi_001
miskiin_002
maqsuura_001
maayoo_001
mrakkib_001
faad_001
idhn_002
fitr_001
firt_001
cilk_001
carya_001
hizhzh_001
taalic_001
gishsh_001
sill_001
ghaadh_002
laasaq_001
kiir_001
miiq_001
diis_001
raddid_001
niir_001
xiir_002
zhiish_001
cilw_001
maasaat_001
idaari_001
sarwal_001
shkuun_002
shnuwwa_002
shniyya_002
tafkiir_001
laqab_001
gafsa_001
msuuhib_001
kraac_001
mitfa

il_bhiira_001
rciiya_001
wqiiya_001
izh_zhanuub_001
btuul_001
kluuf_001
zayyit_001
mzayyit_001
mmakyizh_001
msayyiq_001
mtallaq_001
trash_001
mhaddid_002
is_siin_001
catr_002
tabla_001
gaabsi_001
saahli_002
ghluuq_001
ftuul_001
suxuuna_001
dhruura_001
mruuwa_001
bluuma_001
shkuula_001
zalliq_001
zalqa_001
maadaar_001
qastli_001
qashshaabiiya_001
tbaaci_001
cuud_il_qmaari_001
kaamaal_001
shaataar_001
saaqaat_001
saaqaat_002
tiihaan_001
diiwaan_001
diiwaana_001
taalo_001
lham_ghalmi_001
daar_suuri_001
suuri_002
suuri_003
biit_suuri_001
xubza_suuri_001
tmaatim_suuri_001
qacda_suuri_001
qasriiya_suuri_001
fuuraar_001
mkaatib_suuri_001
buuqaal_001
sabbaala_suuri_001
ftiira_suuri_001
kuudhaar_001
kuukaal_001
hwaanti_001
duulaab_001
difla_001
ciisha_difla_001
liila_difla_001
luubaana_001
ciisha_csal_001
nuushaana_001
kuusaala_001
baaziin_001
kaaliis_001
faariik_001
kaabiis_001
baalac_l_baakiita_001
biiliik_002
buuliish_001
zuubiida_001
quufiiya_001
maacuun_001
daamuus_001
macmil_il_yaazhuur_0

kiif_illi_001
b_zaayd_001
aataar_001
qurbaan_001
kushkaara_001
burgaadha_001
zhurnaata_001
zanziir_001
qandiil_001
dandiiq_001
sarghiin_001
mandiil_001
barniita_001
bandiira_001
antiika_002
blazh_001
infisaam_001
qadd_illi_001
qadd_ma_002
fi_cuudh_001
claa_qadd_ma_claa_qadd_ma_001
claa_ma_001
ghiir_ma_001
raghm_001
ghiir_laa_001
firniiz_001
kuntiil_001
qunfiid_001
muqniin_001
zugdiida_001
qarmuud_001
kantuul_001
thim_001
czhaar_001
xidma_suuri_002
farda_ymiin_001
farda_ysaar_001
murr_ki_d_difla_001
caam_murr_001
muubiilya_qiima_001
haazhaat_qiima_001
tarbuush_001
haa_001
l_illi_001
ca_l_illi_001
biiniin_001
maa_biin_il_biiniin_001
buu_fartiin_001
shlaq_002
ruuh_002
nafs_002
xabbir_001
barqiiya_001
xazhzh_001
fanduur_001
sarsuut_001
ghambuuz_001
qarquush_001
sharyuul_001
anduul_001
gazduur_001
illa_ma_001
bcadh_001
iclaami_001
ghiidh_001
nashiid_watani_001
cammil_001
i_001
ik_001
ha_001
na_001
kum_001
hum_001
u_001
muxtalif_001
intaxab_001
sukna_001
talawwut_001
ahmad_001
muhammad_001
i

In [104]:
SkosConceptScheme.objects.all().delete()

(22, {'vocabs.SkosConceptScheme': 6, 'vocabs.SkosConcept_scheme': 16})

In [105]:
SkosConcept.objects.all().delete()

(231,
 {'vocabs.SkosConcept': 16,
  'vocabs.SkosConcept_label': 0,
  'vocabs.SkosConcept_scheme': 0,
  'vocabs.SkosConcept_skos_broader': 0,
  'vocabs.SkosConcept_skos_broadmatch': 0,
  'vocabs.SkosConcept_skos_closematch': 0,
  'vocabs.SkosConcept_skos_exactmatch': 0,
  'vocabs.SkosConcept_skos_narrower': 0,
  'vocabs.SkosConcept_skos_related': 0,
  'words.Word': 109,
  'words.Word_has_form': 2,
  'words.Word_has_gram': 0,
  'words.Word_has_sense': 104,
  'words.Word_reference': 0})

In [62]:
Word.objects.all().delete()

(263,
 {'words.Word': 147,
  'words.Word_has_form': 7,
  'words.Word_has_gram': 0,
  'words.Word_has_sense': 109,
  'words.Word_reference': 0})